# Playground

The purpose of this notebook is for two things.

1. Explore and examine the TSV file containing the data of locations
2. Implement the algorithms (edit distance, confidence level)
3. Tweak the algorithms (edit distance, confidence level)

In [1]:
import math

In [5]:
def edge_d(a, b):
    '''
    Compute the edit distance between two words using dynamic programming
    Input: Word a, Word b
    Output: Graph of distances. A 2-dimensional array
    '''
    
    # Create graph nxm
    n = len(a)
    m = len(b)
    graph = []
    
    # Edge Case
    if n == 0:
        return m
    elif m == 0:
        return n
    
    # One extra column / row for initial empty state
    n += 1
    m += 1
    
    for i in range(n):
        graph.append([0] * m)
    
    # Call helper with graph
    return helper(a, b, graph)

def helper(a, b, graph):
    
    # Solve smallest subproblem 
    n = len(a) + 1
    m = len(b) + 1
    
    for i in range(n):
        graph[i][0] = i
    
    for i in range(1, m):
        graph[0][i] = i
        
    # Solve subproblems recursively
    for i in range(1, n):
        for j in range(1, m):
            diff = 1 if a[i-1] != b[j-1] else 0
            graph[i][j] = min(graph[i-1][j] + 1, graph[i][j-1] + 1, graph[i-1][j-1] + diff)
    return graph
    
# edge_d('exponential', 'polynomial')
edge_d('sunny', 'sunday')
# edge_d('Calgary', '')
# edge_d('Snow', 'Know')

[[0, 1, 2, 3, 4, 5, 6],
 [1, 0, 1, 2, 3, 4, 5],
 [2, 1, 0, 1, 2, 3, 4],
 [3, 2, 1, 0, 1, 2, 3],
 [4, 3, 2, 1, 1, 2, 3],
 [5, 4, 3, 2, 2, 2, 2]]

In [10]:
import sys
import psycopg2
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

/Users/dougouk/miniconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Create a table called cities

In [117]:
create_table_command = (
    """
    CREATE TABLE cities (
        id TEXT PRIMARY KEY,
        name VARCHAR(200),
        ascii VARCHAR(200),
        alt_name VARCHAR(5000),
        lat FLOAT,
        long FLOAT,
        feature_class VARCHAR(1),
        feature_code VARCHAR(10),
        country_code VARCHAR(2),
        alt_country_code VARCHAR(60),
        admin_1_code VARCHAR(20),
        admin_2_code VARCHAR(20),
        admin_3_code VARCHAR(20),
        admin_4_code VARCHAR(20),
        population BIGINT,
        elevation INTEGER,
        dem INTEGER,
        timezone VARCHAR(40),
        modification_date VARCHAR(10)
    )
    """
)

con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
try:
    cur = con.cursor()
    cur.execute(create_table_command)
    cur.close()
    con.commit()
    print('Successfully created table')
except (Exception, psycopg2.DatabaseError) as error:
    print(error)


### Populate database from TSV File

In [216]:
con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
cur = con.cursor()

with open('cities_canada-usa.tsv','r',encoding='UTF-8') as tsvin:
    next(tsvin)
    cur.copy_from(tsvin, 'cities', sep='\t', null='')
    
con.commit()

### Delete Table Cities

In [205]:
con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
cur = con.cursor()

try:
    cur.execute('DELETE FROM cities')
except (Exception, psycopg2.DatabaseError) as error:
    print(f'Error: {error}')
finally:
    con.commit()
    cur.close()
    con.close()

Error: relation "cities" does not exist
LINE 1: DELETE FROM cities
                    ^



In [8]:
# Create class of data objects

class City:
    def __init__(self, name, lat, long):
        self.name = name
        self.lat = lat
        self.long = long
        self.confidence_level = 0.0
        self.edit_distance = 0
        self.physical_distance = None
        
    def __repr__(self):
        return f'{self.name}  (Confidence {self.confidence_level}) (Edit Distance {self.edit_distance}) (Distance {self.physical_distance})'

In [36]:

EDIT_DISTANCE_THRESHOLD = 3
def search(query, lat=None, long=None):
    '''
    Inputs
        query: name of city
        lat: optional latitude parameter
        longitude: optional longitude parameter
    Output
        Array of cities that may be good results based on the input parameters
    '''
    if query is '':
        return []
    con = None
    con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
    cur = con.cursor()
    # cur.execute('SELECT * FROM cities')
    cur.execute('SELECT * FROM cities WHERE LOWER(cities.name) LIKE \'%' + query.lower() + '%\'')
    rows = cur.fetchall()
    
    similar_cities = []
    for row in rows:
        confidence_graph = edge_d(query.lower(), row[1].lower())
        if query.lower() in row[1].lower() or confidence_graph[-1][-1] < EDIT_DISTANCE_THRESHOLD:
            city = City(row[1], row[4], row[5])
            city.edit_distance = confidence_graph[-1][-1]
            if lat and long:
                city.physical_distance = lat_long_d(lat, long, row[4], row[5])

            calc_confidence_level(city, query)
            similar_cities.append(city)
    cur.close()
    con.close()
    return similar_cities
    
search('Calgary')

[Calgary  (Confidence 1.0) (Edit Distance 0) (Distance None)]

In [30]:
results = search('Calg', 51.012782, -114.3543368)
sorted_search = sorted(results, key=lambda x: x.confidence_level, reverse=True)
sorted_search

[Calgary  (Confidence 0.5588917292165543) (Edit Distance 3) (Distance 19.272830905652228)]

In [15]:
results = search('londo', 43.70011, -79.4163)
sorted_search = sorted(results, key=lambda x: x.confidence_level, reverse=True)
sorted_search

[London  (Confidence 0.8362086332750764) (Edit Distance 1) (Distance 167.188390567362),
 London  (Confidence 0.7813284021986127) (Edit Distance 1) (Distance 539.9778109299942),
 London  (Confidence 0.766086077948264) (Edit Distance 1) (Distance 830.5732255476521),
 New London  (Confidence 0.27542660802558644) (Edit Distance 5) (Distance 653.6943079928908),
 New London  (Confidence 0.2707224618404707) (Edit Distance 5) (Distance 748.9684332217317),
 Londontowne  (Confidence 0.22508475840288877) (Edit Distance 6) (Distance 581.6818345626483),
 Londonderry  (Confidence 0.22055836811672933) (Edit Distance 6) (Distance 657.5350329325814)]

In [37]:
results = search('', None, None)
results

[]

In [13]:
def lat_long_d(_lat_a, _long_a, _lat_b, _long_b):
    '''
    Returns distance between two points in km
    '''
    lat_a = math.radians(_lat_a)
    lat_b = math.radians(_lat_b)
    long_a = math.radians(_long_a)
    long_b = math.radians(_long_b)
    
    # approximate radius of earth in km
    R = 6373.0
    dlon = long_b - long_a
    dlat = lat_b - lat_a

    a = math.sin(dlat / 2)**2 + math.cos(lat_a) * math.cos(lat_b) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

pair_1 = [44.5544837, -78.7165327]
pair_2 = [45.5016889, -73.567256]
lat_long_d(pair_1[0], pair_1[1], pair_2[0], pair_2[1])

418.20130730887263

In [12]:
def calc_confidence_level(city, query_location):
    '''
    Input: City object variable
    Output: Calculated confidence level
    '''
    
    INERTIA = len(query_location) * 2
    confidence_level = 0.0
    if city.physical_distance:
        confidence_level = 0.8 * (INERTIA / (INERTIA + city.edit_distance**2)) + 0.2 * ( 1 / (1 + city.physical_distance / 200))
    else:
        confidence_level = (INERTIA / (INERTIA + city.edit_distance**2))
    city.confidence_level = confidence_level
    return city.confidence_level

In [ ]:
con = None
con = connect(database= 'coveo_cities', user='postgres', host = 'localhost', password='password')
cur = con.cursor()
cur.execute('SELECT * FROM cities')
rows = cur.fetchall()
print(f'Database result size {len(rows)}')
len(rows)